In [13]:
!pip install -U memory_profiler

In [14]:
#2. Hash
import hashlib
from cryptography.hazmat.primitives import hashes

def hash_image_sha3_512(image_path):
    with open(image_path, "rb") as img_file:
        image_data = img_file.read()

    digest = hashes.Hash(hashes.SHA3_512())
    digest.update(image_data)
    image_hash = digest.finalize()

    return image_hash


In [15]:
# 3. Sign the hash of an image

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.backends import default_backend



def sign_image(image_path, private_key):
    # Get the current process
    process = psutil.Process()

    # Record CPU times before function execution
    cpu_times_before = process.cpu_times()

    start_time = time.time()
    gpus_before = GPUtil.getGPUs()
    memory_used_before = sum(gpu.memoryUsed for gpu in gpus_before)
    gpu_load_before = sum(gpu.load for gpu in gpus_before)

    image_hash = hash_image_sha3_512(image_path)
    print("Image hash:", image_hash.hex())

    # Sign the hash with the private key
    signature = private_key.sign(
        image_hash,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )



    end_time = time.time()

    # Record CPU times after function execution
    cpu_times_after = process.cpu_times()
    gpus_after = GPUtil.getGPUs()
    memory_used_after = sum(gpu.memoryUsed for gpu in gpus_after)
    gpu_load_after = sum(gpu.load for gpu in gpus_after)

    memory_diff = memory_used_after - memory_used_before
    gpu_load_diff = gpu_load_after - gpu_load_before
    user_time = cpu_times_after.user - cpu_times_before.user
    system_time = cpu_times_after.system - cpu_times_before.system
    total_time = end_time - start_time

    # Print results
    print(f"User CPU time: {user_time:.4f} seconds")
    print(f"System CPU time: {system_time:.4f} seconds")
    print(f"Total elapsed time: {total_time:.4f} seconds")
    print(f"CPU usage: {(user_time + system_time) / total_time * 100:.2f}%")
    print(f"GPU memory used: {memory_diff:.2f} MB")
    print(f"GPU load change: {gpu_load_diff:.2f}")
    GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)

    return signature

In [16]:
# 4. Verify the signature
def verify_image_signature(image_path, signature, public_key):


    # Get the current process
    process = psutil.Process()

    # Record CPU times before function execution
    cpu_times_before = process.cpu_times()

    start_time = time.time()
    gpus_before = GPUtil.getGPUs()
    memory_used_before = sum(gpu.memoryUsed for gpu in gpus_before)
    gpu_load_before = sum(gpu.load for gpu in gpus_before)


    with open(image_path, "rb") as img_file:
        image_data = img_file.read()

    image_hash = hash_image_sha3_512(image_path)

    # Verify the signature with the public key
    try:
        public_key.verify(
            signature,
            image_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("Signature is valid!")
    except Exception as e:
        print(f"Signature verification failed: {e}")

    # Record CPU times after function execution
    end_time = time.time()

    cpu_times_after = process.cpu_times()
    gpus_after = GPUtil.getGPUs()
    memory_used_after = sum(gpu.memoryUsed for gpu in gpus_after)
    gpu_load_after = sum(gpu.load for gpu in gpus_after)

    memory_diff = memory_used_after - memory_used_before
    gpu_load_diff = gpu_load_after - gpu_load_before
    user_time = cpu_times_after.user - cpu_times_before.user
    system_time = cpu_times_after.system - cpu_times_before.system
    total_time = end_time - start_time

    # Print results
    print(f"User CPU time: {user_time:.4f} seconds")
    print(f"System CPU time: {system_time:.4f} seconds")
    print(f"Total elapsed time: {total_time:.4f} seconds")
    print(f"CPU usage: {(user_time + system_time) / total_time * 100:.2f}%")
    print(f"GPU memory used: {memory_diff:.2f} MB")
    print(f"GPU load change: {gpu_load_diff:.2f}")
    GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)

# Load signature and verify
#with open("image_signature.sig", "rb") as sig_file:
#    saved_signature = sig_file.read()

#verify_image_signature(image_path, saved_signature, public_key)

In [17]:
!pip install GPUtil

In [18]:
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.backends import default_backend
import time
import psutil
import GPUtil

def main():
    # Example for image hash
    image_path = "Capture.JPG"

    # Load the private key from the file
    with open("private_key.pem", "rb") as key_file:
        private_key = load_pem_private_key(
            key_file.read(),
            password=None,  # Add a password here if the private key is encrypted
            backend=default_backend()
        )

    with open("public_key.pem", "rb") as key_file:
        public_key = load_pem_public_key(
          key_file.read(),
          backend=default_backend()
        )

    start_time = time.time()
    hashed = hash_image_sha3_512("Capture.JPG")

    signature = sign_image(image_path, private_key)

    # Save signature to a file
    with open("image_signature.sig", "wb") as sig_file:
        sig_file.write(signature)

    print("Image signed successfully. Signature saved to 'image_signature.sig'.")

    # Load signature from the file
    with open("image_signature.sig", "rb") as sig_file:
        sign = sig_file.read()

    verify = verify_image_signature(image_path, sign, public_key)

    #print("Hash value:", hash_value)
    print("--- %s seconds ---" % (time.time() - start_time))

In [19]:
main()

Image hash: 7ad424763b2633c196ee348da36a32c31ebaa35e430f008761ac45b885e0eaf73b0447b0d951793910ddd1a7aba50ef69d9df5f5aa4ed435555ba1e9b7fe7625
User CPU time: 0.0000 seconds
System CPU time: 0.0000 seconds
Total elapsed time: 0.0547 seconds
CPU usage: 0.00%
GPU memory used: 0.00 MB
GPU load change: 0.00
| ID | GPU | MEM |
------------------
|  0 | 29% | 12% |
Image signed successfully. Signature saved to 'image_signature.sig'.
Signature is valid!
User CPU time: 0.0156 seconds
System CPU time: 0.0000 seconds
Total elapsed time: 0.0426 seconds
CPU usage: 36.68%
GPU memory used: -8.00 MB
GPU load change: 0.00
| ID | GPU | MEM |
------------------
|  0 | 29% | 12% |
--- 0.29016661643981934 seconds ---
